*(Notebook hecho por Victoria)*

### DATA PT12 - Proyecto final - Grupo 2
#### Una notebook para todo. 
##### Solo falta resolver lo de las dummies

In [51]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [52]:
# leo el archivo parquet datos crudos
# How to read a Parquet file into Pandas DataFrame?
gm_sitios = pd.read_parquet("datos/gm_raw_parquet/gm_sitios_raw.parquet" , engine='fastparquet')

In [55]:
# Eliminamos columnas que no serán utilizadas
# NO ELIMINO 'price', 'MISC', PUEDE SERVIR PARA LAS RECOMENDACIONES
# Por ahora conservo latitude y longitude porque puede servirnos para mostrar locales en el dashboard

gm_sitios = gm_sitios.drop(columns=['description', 'state', 'relative_results', 'url'])

In [57]:
# selecciono las filas que tienen pizza restaurant
# df_mt_sitios_pizza = df_mt_sitios[df_mt_sitios['category'].apply(lambda x: 'Pizza restaurant' in x)]
# TypeError: argument of type 'NoneType' is not iterable

gm_sitios_pizza = gm_sitios[gm_sitios['category'].apply(lambda x: isinstance(x, list) and 'Pizza restaurant' in x)]

In [59]:
# Extract state (two uppercase letters) using regex
gm_sitios_pizza['state'] = gm_sitios_pizza['address'].str.extract(r',\s*([A-Z]{2})\s*\d{5}') 

In [61]:
# selecciono las pizzerias de NJ y NY
gm_sitios_pizza_NJNY = gm_sitios_pizza[gm_sitios_pizza['state'].isin(['NJ', 'NY'])]

In [62]:
# en algunas direcciones dice ', United States' al final / Remove ', United States' if present
gm_sitios_pizza_NJNY['cleaned_address'] = [addr.replace(", United States", "") for addr in gm_sitios_pizza_NJNY['address']]

In [63]:
# Regex pattern to extract street address, city, and ZIP code
pattern_1 = r'(?P<street_address_temp>.+),\s*(?P<city>[^,]+),\s*[A-Z]{2}\s*(?P<zip_code>\d{5})$'

# Aplicar regex a la columna 'address'
df_extracted = gm_sitios_pizza_NJNY['cleaned_address'].str.extract(pattern_1)

# Combinar con el DataFrame original
gm_sitios_pizza_NJNY = gm_sitios_pizza_NJNY.join(df_extracted)

In [65]:
# To capture only the part to the right of the comma
gm_sitios_pizza_NJNY["street_address"] = gm_sitios_pizza_NJNY["street_address_temp"].str.split(",", n=1).str[1].str.strip()

In [67]:
# elimino las columnas que ya no usamos
# df.drop(['Column1', 'Columns2'], axis=1)
gm_sitios_pizza_NJNY.drop(['address', 'cleaned_address', 'street_address_temp'], axis=1, inplace=True)

In [68]:
# elimino los duplicados manteniendo la primera instancia
gm_sitios_pizza_NJNY.drop_duplicates(subset=['gmap_id'], keep='first', inplace=True)

In [69]:
# Convert to dictionary format para extraer los horarios por dia de cada local. La columna es una lista de listas
gm_sitios_pizza_NJNY['hours_dict'] = gm_sitios_pizza_NJNY['hours'].apply(lambda x: dict(x) if isinstance(x, list) else {})

In [70]:
# Expand 'hours_dict' into separate columns (one per day)
hours_expanded = gm_sitios_pizza_NJNY['hours_dict'].apply(pd.Series)

In [71]:
# Merge expanded columns back into original DataFrame
gm_sitios_pizza_NJNY = pd.concat([gm_sitios_pizza_NJNY, hours_expanded], axis=1)

In [74]:
# reemplazar '₩', '₩₩' por $ o $$ - esta forma de hacerlo debe ser muy lenta
#replacements = str.maketrans({"h": "H", "e": "E", "o": "O"})
#res = s.translate(replacements)
# string keys in translate table must be of length 1
gm_sitios_pizza_NJNY['price'] = (
    gm_sitios_pizza_NJNY['price']
    .str.replace("₩₩", "$$", regex=False)  # Primero reemplaza ₩₩ por $$
    .str.replace("₩", "$", regex=False)    # Luego reemplaza ₩ por $
)

In [76]:
# completo con un diccionario vacios las filas que tienen la columna MISC en NaN
gm_sitios_pizza_NJNY['MISC'] = gm_sitios_pizza_NJNY['MISC'].apply(lambda x: {} if pd.isnull(x) else x)

In [77]:
# Expand 'MISC' into separate columns (one per iem)
MISC_expanded = gm_sitios_pizza_NJNY['MISC'].apply(pd.Series)

In [79]:
# Merge expanded columns back into original DataFrame
# creo que se concatena por los indices. Por eso no necesita que haya una columna en comun
gm_sitios_pizza_NJNY = pd.concat([gm_sitios_pizza_NJNY, MISC_expanded], axis=1)

In [81]:
# df.drop(['Column1', 'Columns2'], axis=1)
# elimino columnas que ya no necesito
gm_sitios_pizza_NJNY.drop(['category', 'MISC', 'hours', 'hours_dict', 'Offerings', 'Dining options', 'Health & safety', 'Accessibility', 'Crowd', 'Planning', 
'Payments', 'Highlights', 'From the business', 'Health and safety'], axis=1, inplace=True)

In [82]:
# por ahora elimino tambien las columnas que luego usare para dummies porque no puedo seleccionarlas 
# cuando separo las filas de sitios con drop.duplicate porque son listas
gm_sitios_pizza_NJNY.drop(['Service options', 'Amenities', 'Atmosphere', 'Popular for'], axis=1, inplace=True)

In [84]:
# Leo los datasets de reviews completos
df_rev_NJ = pd.read_parquet('datos/gm_raw_parquet/gm_review_NJ_raw.parquet' , engine='fastparquet')
df_rev_NY = pd.read_parquet('datos/gm_raw_parquet/gm_review_NY_raw.parquet' , engine='fastparquet')

In [86]:
# Eliminamos, por ahora, name, pics y resp. no usaremos esas columnas
df_rev_NJ = df_rev_NJ.drop(columns=['name', 'pics', 'resp'])
df_rev_NY = df_rev_NY.drop(columns=['name', 'pics', 'resp'])

In [87]:
# eliminar los duplicados manteniendo la primera instancia
df_rev_NJ = df_rev_NJ.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')
df_rev_NY = df_rev_NY.drop_duplicates(subset=['user_id', 'gmap_id', 'time'], keep='first')

In [88]:
# convertir columna time en int64 a datetime
df_rev_NJ['date'] = pd.to_datetime(df_rev_NJ['time'], unit='ms')
df_rev_NY['date'] = pd.to_datetime(df_rev_NY['time'], unit='ms')

# Eliminamos la columan time
df_rev_NJ = df_rev_NJ.drop(columns=['time'])
df_rev_NY = df_rev_NY.drop(columns=['time'])

In [89]:
# uno los dataframes de reviews hacer join con el dataframe de sitios /
# asi solo conservare los locales para los que haya reviews y las reviews para las que conozcamos los locales
df_rev_NJNY = pd.concat([df_rev_NJ, df_rev_NY], ignore_index=True)

In [90]:
# merge entre los dos dataframe . Luego guardare solo los que tengan datos en ambos.
gm_NJNY = pd.merge(gm_sitios_pizza_NJNY, df_rev_NJNY, how='inner', on='gmap_id') # tengo que hacer inner join

In [93]:
# vuelvo a separar los archivos para dejarlos listos para la base de datos
# unique_age_city = df[['Age', 'City']].drop_duplicates()
# por ahora no seleccionamos las columnas 'Service options', 'Amenities', 'Atmosphere', 'Popular for' porque las listas no permite drop.duplicates() 
gm_sitios_NJNY = gm_NJNY[['gmap_id', 'name', 'street_address', 'city', 'state', 'zip_code',   'latitude','longitude','avg_rating','num_of_reviews','price',
                   'Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', ]].drop_duplicates()

In [96]:
# ahora selecciono las reviews
gm_rev_NJNY = gm_NJNY[['gmap_id', 'user_id', 'date', 'rating' , 'text']]

In [ ]:
# estos son los archivos que hay que cargar en las tablas para dashboard y modelos de ML

# guardo gm_sitios_NJNY
gm_sitios_NJNY.to_parquet('datos/gm_raw_parquet/gm_sitios_NJNY_202502182030.parquet' , engine='fastparquet')

# guardo gm_rev_NJNY
gm_rev_NJNY.to_parquet('datos/gm_raw_parquet/gm_rev_NJNY_202502182030.parquet' , engine='fastparquet')